<a href="https://colab.research.google.com/github/Roberto014/Anal-tica-de-datos-y-herramientas-de-inteligencia-artificial-II/blob/main/Copia_de_Dashboard_para_mi_ciudad_TODO_INCLUIDO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
%pip install streamlit

In [ ]:
%pip install plotly

In [ ]:
%pip install funpymodeling

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.

In [ ]:
#Instalamos el tunnel local (node.js)
#Para instalar npm en visual studio
#1. Desde google escribir node.js
#2. Instalar la versión mas recomendada
!npm install localtunnel


added 22 packages, and audited 23 packages in 3s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [75]:
%%writefile app.py
# app.py
# Importamos librerías
import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np
from funpymodeling.exploratory import freq_tbl
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# Definimos la instancia de streamlit
@st.cache_resource
# Creamos la función de carga de datos
def load_data_Mexico():
    # Carga desde un archivo csv con índice
    data = pd.read_csv('Datos_limpios (4).csv', encoding='utf-8', index_col=0)

    # Etapa de procesamiento de Datos
    data_cuant = data.copy()

    # Se eliminan columnas innecesarias
    data_cuant.drop(['last_scraped', 'source', 'name', 'description', 'host_name',
                      'host_since', 'host_location', 'host_response_time',
                      'host_verifications', 'host_has_profile_pic',
                      'host_identity_verified', 'neighbourhood_cleansed',
                      'property_type', 'bathrooms_text', 'amenities',
                      'has_availability', 'calendar_last_scraped',
                      'first_review', 'last_review', 'host_is_superhost',
                      'room_type', 'instant_bookable'], axis=1, inplace=True)

    # Seleccionar solo columnas numéricas
    data_cuant = data_cuant.select_dtypes(include=[np.number])

    # Encontramos todos los coeficientes de correlación entre las variables
    corr_factors = abs(data_cuant.corr())

    # Variables a analizar
    data_results = data_cuant[['host_response_rate', 'host_acceptance_rate',
                                'host_total_listings_count', 'accommodates',
                                'reviews_per_month', 'price']]

    # Selección de columnas numéricas del dataframe
    numeric_data = data_cuant.select_dtypes(['float', 'int'])
    numeric_cols = numeric_data.columns

    # Agregamos el DataFrame original para análisis de variables categóricas
    data_cat = data.copy()

    return data, data_cuant, corr_factors, data_results, numeric_data, numeric_cols, data_cat

# Cargo los datos obtenidos de la función 'load_data'
data, data_cuant, corr_factors, data_results, numeric_data, numeric_cols, data_cat = load_data_Mexico()

# Funciones para modelos No Lineales
def funcCuad(x, a, b, c):
    return a * x**2 + b * x + c

def funcExp(x, a, b, c):
  return a*np.exp(-b*x) + c

def func_lineal_producto_coeficientes(x, a, b, c):
  return (a*x**2 + b)/ c*x

# Página principal - Titulo
st.title('ANÁLISIS DE DATOS')

# Selección de análisis
analysis_type = st.sidebar.radio("Seleccione el tipo de análisis",
                                   ("Análisis de Correlación", "Análisis de Regresión Multiple", "Regresión No Lineal", "Análisis Univariado de Variables Categóricas", ))




# Análisis de Correlación
if analysis_type == "Análisis de Correlación":
    st.header('Análisis de Correlación')

    # Checkbox para mostrar el heatmap
    heatmap_checkbox = st.checkbox(label="Mostrar Heatmap")

    if heatmap_checkbox:
        # Generamos un Heat Map para todas las variables
        heatmap = px.imshow(corr_factors, text_auto=True,
                            title="Este es el Heatmap", width=800, height=600)
        st.plotly_chart(heatmap)

    # Checkbox para mostrar la matriz de correlación
    matrix_checkbox = st.checkbox(label="Mostrar matriz de correlación")

    if matrix_checkbox:
        # Mostramos la matriz de correlación
        st.write(corr_factors)


elif analysis_type == "Análisis de Regresión Multiple":

  st.sidebar.subheader('Variables Independientes')
  y_var = st.sidebar.multiselect('Selecciona las variables independientes a incluir:', options=numeric_cols)
  st.sidebar.subheader('Variable Dependiente')
  x_var = st.sidebar.selectbox(label='Seleccione Variable X', options=numeric_cols)
  if y_var and x_var:
        # Preparación de los datos
        X = numeric_data[y_var]  # Usar el DataFrame filtrado
        y = numeric_data[x_var]   # Usar el DataFrame filtrado

        # Separar en conjunto de entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # Crear el modelo de regresión lineal múltiple
        modelo = LinearRegression()

        # Entrenar el modelo
        modelo.fit(X_train, y_train)

        # Realizar predicciones en el conjunto de prueba
        y_pred = modelo.predict(X_test)

        # Calcular métricas
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        correlation_coef = np.corrcoef(y_test, y_pred)[0, 1]  # Coeficiente de correlación

        #Mostrar resultados

        st.write('### Resultados del Análisis de Regresión Múltiple:')
        st.write('**Coeficiente de Determinación (R²):**', r2)
        st.write('**Raíz del Error Cuadrático Medio (RMSE):**', rmse)
        st.write('**Coeficiente de Correlación:**', correlation_coef)



        # Gráfico de dispersión
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, color='blue', label='Predicciones', alpha=0.5)
        plt.scatter(y_test, y_test, color='red', label='Valores Reales', alpha=0.5)  # Añadir valores reales
        plt.xlabel('Valores Reales')
        plt.ylabel('Valores Predichos')
        plt.title('Gráfico de Dispersión de Valores Reales vs Predicciones')
        plt.legend()
        st.pyplot(plt)

        # Calcular la matriz de correlación
        correlation_matrix = numeric_data.corr()
        st.write('### Matriz de Correlación:')
        st.write(correlation_matrix)










# Análisis de Regresión No Lineal
elif analysis_type == "Regresión No Lineal":
    st.header('Análisis de Regresión No Lineal')

    # Selección de variables para graficar
    vars_result_selected = st.selectbox(label='Variable Objetivo', options=data_results.columns)

    st.sidebar.subheader('Variable Independiente')
    x_var = st.sidebar.selectbox(label='Seleccione Variable X', options=numeric_cols)

    st.sidebar.subheader('Función para Graficar')
    func = st.sidebar.selectbox(label='Seleccione Funcion para graficar', options=['Cuadratica', 'Exponencial','func_lineal_producto_coeficientes'])

    if func == 'Cuadratica':
        data_copy = data_cuant.copy()

        # Declaramos las variables dependientes e independientes para la regresión No Lineal
        x = data_cuant[[str(x_var)]]
        y = data_cuant[[str(vars_result_selected)]]

        st.text('Función Cuadrática - y = ax^2 + bx + c')

        try:
            # Ajustamos los parámetros de la función curve_fit
            parametros, covs = curve_fit(funcCuad, data_cuant[x_var], data_cuant[vars_result_selected], maxfev=5000)

            # Predicciones
            y_pred = funcCuad(x, *parametros)
            data_copy['y_pred'] = y_pred

            # Calculamos el coeficiente de determinación del modelo
            r2 = r2_score(y, y_pred)

            # Calculamos el coeficiente de correlación del modelo
            r = np.sqrt(abs(r2))

            # Mostramos el Coeficiente
            st.text('Coeficiente de determinacion (R^2):')
            st.text(str(r2))
            st.text('Coeficiente de Correlación:')
            st.text(str(r))

            # SCATTERPLOT
            # Despliegue de un scatter plot, definiendo las variables "X" y "Y"
            figureRNL = px.scatter(data_frame=data_copy, x=x_var, y=[vars_result_selected, 'y_pred'],
                                    title="Regresión No Lineal - Cuadrática")
            st.plotly_chart(figureRNL)
        except Exception as e:
            st.text('No se puede graficar la grafica ' + str(e))

    if func == 'Exponencial':
        data_copy = data_cuant.copy()
        # Declaramos las variables dependientes e independientes para la regresión No Lineal
        x = data_cuant[[str(x_var)]]
        y = data_cuant[[str(vars_result_selected)]]

        st.text('Función Exponencial  y = a*exp(-bx) + c')
        try:
            parametros, covs = curve_fit(funcExp, data_cuant[x_var], data_cuant[vars_result_selected], maxfev=5000)

            # Predicciones
            y_pred = funcExp(x, *parametros)
            data_copy['y_pred'] = y_pred

            # Calculamos el coeficiente de determinación del modelo
            r2 = r2_score(y, y_pred)

            # Calculamos el coeficiente de correlación del modelo
            r = np.sqrt(abs(r2))

            # Mostramos el Coeficiente
            st.text('Coeficiente de determinacion (R^2):')
            st.text(str(r2))
            st.text('Coeficiente de Correlación:')
            st.text(str(r))

            # SCATTERPLOT
            # Despliegue de un scatter plot, definiendo las variables "X" y "Y"
            figureRNL = px.scatter(data_frame=data_copy, x=x_var, y=[vars_result_selected, 'y_pred'],
                                    title="Regresión No Lineal - Exponencial")
            st.plotly_chart(figureRNL)
        except Exception as e:
            st.text('No se puede graficar la gráfica ' + str(e))

    if func == 'func_lineal_producto_coeficientes':
          data_copy = data_cuant.copy()
          # Declaramos las variables dependientes e independientes para la regresión No Lineal
          x = data_cuant[[str(x_var)]]
          y = data_cuant[[str(vars_result_selected)]]
          st.text('Función Lineal con producto de coeficientes  Función y = (a*x^2 + b) / (c*x)')
          try:
              parametros, covs = curve_fit(func_lineal_producto_coeficientes, data_cuant[x_var], data_cuant[vars_result_selected], maxfev=5000)
              # Predicciones
              y_pred = func_lineal_producto_coeficientes(x, *parametros)
              data_copy['y_pred'] = y_pred
              # Calculamos el coeficiente de determinación del modelo
              r2 = r2_score(y, y_pred)
              # Calculamos el coeficiente de correlación del modelo
              r = np.sqrt(abs(r2))
              # Mostramos el Coeficiente
              st.text('Coeficiente de determinacion (R^2):')
              st.text(str(r2))
              st.text('Coeficiente de Correlación:')
              st.text(str(r))
              # SCATTERPLOT
              # Despliegue de un scatter plot, definiendo las variables "X" y "Y"
              figureRNL = px.scatter(data_frame=data_copy, x=x_var, y=[vars_result_selected, 'y_pred'],
                                    title="Regresión No Lineal - Exponencial")
              st.plotly_chart(figureRNL)
          except Exception as e:
              st.text('No se puede graficar la gráfica ' + str(e))



elif analysis_type == "Análisis Univariado de Variables Categóricas":
    st.header('Análisis Univariado de Variables Categóricas')

    # Selección de variable categórica
    categorical_var = st.selectbox('Selecciona una variable categórica:', data_cat.select_dtypes(include=['object']).columns)

    if categorical_var:
        # Filtramos las categorías de la variable seleccionada
        categories = data_cat[categorical_var].value_counts().index.tolist()

        # Selección de categorías
        selected_categories = st.multiselect('Selecciona las categorías a incluir:', categories)

        # Selección del tipo de gráfico
        chart_type = st.selectbox('Selecciona el tipo de gráfico:', ['Bar', 'Pie'])

        if selected_categories:
            # Filtramos los datos por las categorías seleccionadas

            filtered_data = data_cat[data_cat[categorical_var].isin(selected_categories)]

            # Realizamos el análisis univariado
            table1 = freq_tbl(filtered_data[categorical_var])
            table2 = table1.drop(['percentage','cumulative_perc'], axis=1)

            # Ajusto el índice de mi dataframe
            filtro_index = table2.set_index(categorical_var)

            # Gráfico de barras o pastel
            if chart_type == 'Pie':
                plt.figure(figsize=(6,6))
                plt.title(categorical_var)
                filtro_index["frequency"].plot(kind='pie', autopct='%1.1f %%')
                st.pyplot(plt)
            elif chart_type == 'Bar':
                plt.figure(figsize=(10,5))
                plt.title(categorical_var)
                filtro_index["frequency"].plot(kind='bar')
                st.pyplot(plt)


Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.105.20.134



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.105.20.134:8501

your url is: https://stupid-ads-open.loca.lt
